In [31]:
import numpy as np
from scipy.io import loadmat
import random
from sklearn.metrics import confusion_matrix as cm

df = loadmat('data5.mat')
data = df['x']
np.random.shuffle(data)

def init_data(data):
    X = np.array(data[:2148, :-1], dtype = float)
    y = np.array(data[:2148, -1], dtype = int)
    X = (X - X.mean(axis = 0))/X.std(axis = 0)
    return X, y

#Gaussian Kernel
def gaussian(x,center,sigma,beta):
    return np.exp(-beta * (np.linalg.norm(x - center)) ** 2)
#Multi Quadratic Kernel
def multi_quadric(x, center, sigma, beta):
    return ((np.linalg.norm(x - center)) ** 2 + sigma ** 2) ** 0.5
#Linear Kernel
def linear(x, center, sigma, beta):
    return np.linalg.norm(x - center)

X, y = init_data(data)
train_X = X[ : 1600]
train_y = y[ : 1600]
test_X = X[1600 : 2148]
test_y = y[1600 : 2148]

def fit_rbf(train_X, train_y, test_X, test_y):
    centers, labels = kmeans(train_X, K=550)

    sigma = np.zeros((len(centers), 1))
    beta = np.zeros((len(centers), 1))
    cluster_size = np.zeros((len(centers), 1))

    for i in range(len(train_X)):
        sigma[labels[i]] += np.linalg.norm(train_X[i] - centers[labels[i]])
        cluster_size[labels[i]] += 1

    sigma = sigma / cluster_size
    beta = (1.0 / 2) * (sigma * sigma + 1e-6)

    H = np.zeros((len(train_X), len(centers)))

    for i in range(len(train_X)):
        for j in range(len(centers)):
            H[i, j] = linear(train_X[i], centers[j], sigma[j], beta[j])

    W = np.dot(np.linalg.pinv(H), train_y)

    #Testing
    H_test = np.zeros([len(test_X), len(centers)])
    for i in range(len(test_X)):
        for j in range(len(centers)):
            H_test[i, j] = linear(test_X[i], centers[j], sigma[j], beta[j])

    y_pred = np.dot(H_test, W)
    for i in range(len(y_pred)):
        y_pred[i] = 1 if y_pred[i]>=0.5 else 0
        
    accuracy = 0    
    for i in range(len(y_pred)):
        if y_pred[i] == test_y[i]:
            accuracy +=1
    accuracy /= len(y_pred)
    print(accuracy)
    return y_pred, accuracy

def kmeans(X,K=3,max_iter=4000):
    m,n = np.shape(X)
    def create_dict(n):
        a = {}
        for i in range(n):
            a[f'{i}'] = []
        return a 
    def calc_distance(A,B):
       return np.linalg.norm(A-B,2)
    
    def return_cluster(c_ind,ind):
        for i in c_ind.keys():
            if ind in c_ind[i]:
                return int(i)


    #initialize centers
    index_random = np.random.randint(0,m,K)
    c = []
    for i in range(K):
        c.append(X[index_random[i]])
    #update centers
    c1 = np.zeros((K,n))
    st = False
    iter = 0
    while True:
        if iter == max_iter or st:
            break   
        c_ind = create_dict(K)
        for i in range(m):
            d = []
            for j in range(K):
                d.append(calc_distance(X[i],c[j]))
            index = np.argmin(d)
            c_ind[f'{index}'].append(i) 
        for i in range(K):
            c1_ind = c_ind[f'{i}']
            if len(c1_ind):
                for j in range(len(c1_ind)):
                    c1[i] += X[c1_ind[j]]
                c1[i] /= len(c1_ind)
        if (c==c1).all():
            st = True
        
        c = c1
        iter += 1

    labels = []
    for i in range(m):
        labels.append(return_cluster(c_ind,i))
    return c,labels

y_pred, _ = fit_rbf(train_X, train_y, test_X, test_y)
for i in range(len(y_pred)):
    y_pred[i] = 1 if y_pred[i] > 0.5 else 0

confmat = cm(test_y, y_pred)

Accuracy = (confmat[0][0]+confmat[1][1])/len(y_pred)
Sensitivity = (confmat[1][1])/(confmat[1][0] + confmat[1][1])
Specificity = (confmat[0][0])/(confmat[0][0] + confmat[0][1])

print(confmat)
print('\nAccuracy: ', Accuracy, 'sensitivity: ', Sensitivity,'specificity: ',Specificity,'\n')

av_acc = 0

#k-fold
for k in range(5):
    X = X_tot[0 : 1718]
    y = y_tot[0 : 1718]
    X_val = X_tot[1718 :]
    y_val = y_tot[1718 :]
    print('Fold', k+1,'Acc: ')
    _, acc = fit_rbf(X, y, X_val, y_val)
    av_acc += acc
    X_tot[0 : 430] = X_val
    X_tot[430 : ] = X
    y_tot[0 : 430] = y_val
    y_tot[430 : ] = y

av_acc /= 5
print('\nAverage Accuracy: ',av_acc)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in true_divide


0.958029197080292
[[272  14]
 [  9 253]]

Accuracy:  0.958029197080292 sensitivity:  0.9656488549618321 specificity:  0.951048951048951 

Fold 1 Acc: 
0.9348837209302325
Fold 2 Acc: 
0.9604651162790697
Fold 3 Acc: 
0.9604651162790697
Fold 4 Acc: 
0.9441860465116279
Fold 5 Acc: 
0.9511627906976744

Average Accuracy:  0.9502325581395349
